<a href="https://colab.research.google.com/github/AICFT-Miyata/AICFT/blob/main/semantic_search_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# --- 0. 必要なライブラリのインストールとインポート ---
# 警告が出ますが、無視して処理を続行してください。

# 1. 意味検索と言語処理用ライブラリ
!pip install -qq fugashi ipadic unidic-lite sentence-transformers scikit-learn gspread

# 2. GGUF実行に必要なllama-cpp-pythonをGPUサポート付きでインストール
# ⚠️ インストールには時間がかかり、途中文字化けのような出力が出ますが、処理を中断しないでください。
!CMAKE_ARGS="-DLLAMA_CUDA=on" FORCE_CMAKE=1 pip install -qq llama-cpp-python

# --- ライブラリのインポート ---
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gspread
import time
import google.auth
from google.colab import auth

# GGUF用ライブラリ（ここでLlamaクラスを定義）
try:
    from llama_cpp import Llama
    print("✅ Llama-cpp-pythonのインポートに成功しました。")
except ImportError:
    # インストールが成功していれば、このエラーは出ないはず
    print("❌ Llama-cpp-pythonのインポートに失敗しました。AI洗練機能は動作しません。")
    Llama = None # エラーハンドリングのためLlamaをNoneに設定

# --- 1. 定数・初期設定 ---
# ⚠️ 以下の3つのパス・名前をあなたの環境に合わせて最終修正してください
SPREADSHEET_NAME = 'スプレッドシート作成検証'
TARGET_SHEET = '検索入力シート'
TRAINING_DATA_PATH = '/content/drive/MyDrive/AICFT/DB.csv'

# ⚠️ GGUFモデルのパスを、あなたがアップロードした Q5_K_M のものに修正してください
GGUF_MODEL_PATH = '/content/drive/MyDrive/AICFT/LLM_GGUF/llama3-8b-instruct-Q5_K_M.gguf'

# モデルのロード (意味検索用)
EMBEDDING_MODEL = SentenceTransformer('cl-tohoku/bert-base-japanese-whole-word-masking')

# GGUFモデルの初期設定 (AI洗練用)
LLAMA_CLIENT = None
if Llama:
    try:
        # モデルのロード（メモリ効率が良いQ5_K_MをGPUの全レイヤーで実行）
        LLAMA_CLIENT = Llama(
            model_path=GGUF_MODEL_PATH,
            n_ctx=4096,
            n_gpu_layers=-1,
            verbose=False
        )
        print("✅ GGUFモデルのロードに成功しました。")
    except Exception as e:
        print(f"❌ GGUFモデルのロードに失敗しました。AI洗練機能は動作しません。エラー: {e}")


# --- 2. データの準備とエンベディング生成 ---
def load_and_embed_data(file_path):
    # ... (既存のコード。省略) ...
    try:
        df_train = pd.read_csv(file_path, encoding='utf-8', sep=',',
             usecols=['階級', '目標（50～100文字）', '自己申告（50～100文字）', '上司コメント（50～100文字）'])
        df_train.columns = ['Rank', 'Goal', 'SelfReport', 'SupervisorComment']
        df_train['CombinedText'] = df_train['Goal'].astype(str) + " [SEP] " + df_train['SelfReport'].astype(str)
        embeddings = EMBEDDING_MODEL.encode(df_train['CombinedText'].tolist(), convert_to_tensor=True)
        return df_train, embeddings.cpu().numpy()
    except Exception as e:
        print(f"データ処理エラー: {e}")
        return None, None

df_train, train_embeddings = load_and_embed_data(TRAINING_DATA_PATH)

# --- 3. 検索ロジック ---
def find_best_match_comment(input_goal, input_self_report):
    # ... (既存のコード。省略) ...
    if df_train is None:
        return "ERROR: Training data not loaded.", 0.0
    new_combined_text = str(input_goal) + " [SEP] " + str(input_self_report)
    new_embedding = EMBEDDING_MODEL.encode(new_combined_text, convert_to_tensor=True).cpu().numpy().reshape(1, -1)
    similarities = cosine_similarity(new_embedding, train_embeddings)[0]
    best_match_index = np.argmax(similarities)
    best_comment = df_train.iloc[best_match_index]['SupervisorComment']
    return best_comment, similarities[best_match_index]


# --- 5. AIによるコメント洗練ロジック (GGUF実行) ---
def refine_comment_with_ai(input_goal, input_self_report, best_comment):
    # ... (LLAMA_CLIENT is None のチェックは省略) ...

    try:
        print("--- AIによるコメント洗練開始 (LLaMA 3 GGUF) ---")

        # 修正：日本語での出力形式を強力に指定
        system_prompt = "あなたは人事評価の上司コメントを作成するAIです。日本語で、具体的で建設的なフィードバックを、簡潔に作成してください。"

        user_prompt = f"""
        [入力データ]
        目標: {input_goal}
        自己申告: {input_self_report}
        [推薦コメント (参考)]
        {best_comment}

        上記のデータを参照し、日本語で上司コメントを生成してください。出力はコメント本文のみとします。
        """

        # 修正：応答開始を「日本語のコメント」の形式で始めるよう指示
        prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot|><|start_header_id|>user<|end_header_id|>\n{user_prompt}<|eot|><|start_header_id|>assistant<|end_header_id|>\n**【上司コメント】**"
        # ↑ 応答の開始を明確に日本語の形式で始めるよう強制します

        # 推論実行
        output = LLAMA_CLIENT.create_completion(
            prompt=prompt,
            max_tokens=256,
            temperature=0.4, # 創造性を少し下げ、指示に忠実になるように調整
            stop=["<|eot|>"]
        )

        # 応答から強制開始文字列を除去
        refined_comment = output['choices'][0]['text'].replace('**【上司コメント】**', '').strip()
        print("AI洗練コメント生成成功。")
        return refined_comment

    except Exception as e:
        print(f"AI洗練処理エラー。通常通り検索結果を出力します。エラー: {e}")
        return best_comment


# --- 4. メイン処理 (Apps Scriptから実行される想定) ---
def main_search_and_write():
    if df_train is None:
        print("検索スキップ: 訓練データなし")
        return "検索スキップ: 訓練データなし", 0.0

    print("--- gspread認証開始 ---")
    auth.authenticate_user()

    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)

    spreadsheet = gc.open(SPREADSHEET_NAME)
    input_sheet = spreadsheet.worksheet(TARGET_SHEET)

    data_list = input_sheet.get('B1:E1', value_render_option='UNFORMATTED_VALUE')

    if not data_list or len(data_list[0]) < 4:
        print("ERROR: スプレッドシートのB1～E1範囲にデータが不足しています。処理を中断します。")
        return "ERROR: データ不足", 0.0

    input_data = data_list[0]
    input_goal = input_data[0]
    input_self_report = input_data[3]

    print(f"入力データ: 目標='{input_goal}', 自己申告='{input_self_report}'")

    # 1. 意味検索実行
    best_comment, similarity_score = find_best_match_comment(input_goal, input_self_report)
    print(f"検索結果 (原案): {best_comment}")

    # 2. AI洗練 (LLM連携)
    final_comment = refine_comment_with_ai(input_goal, input_self_report, best_comment)

    # 3. 結果をスプレッドシートに出力
    input_sheet.update('F1', [[final_comment]])
    input_sheet.update('G1', [[f"類似度: {similarity_score:.4f}"]])
    print(f"最終結果 (F1) を出力完了。類似度: {similarity_score:.4f}")

    return final_comment, similarity_score

if __name__ == "__main__":
    # 単体テストとしてメイン処理を実行
    main_search_and_write()



✅ Llama-cpp-pythonのインポートに成功しました。


llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


✅ GGUFモデルのロードに成功しました。
--- gspread認証開始 ---
入力データ: 目標='積極的にワークライフバランスに努める', 自己申告='テレワークを５回実施した。'
検索結果 (原案): 科学的なアプローチによる隊員能力の維持向上に貢献した。プログラムの継続的な効果測定と、他の部隊への展開を検討すべき。
--- AIによるコメント洗練開始 (LLaMA 3 GGUF) ---
AI洗練コメント生成成功。


/tmp/ipython-input-1693090277.py:164: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  input_sheet.update('F1', [[final_comment]])
/tmp/ipython-input-1693090277.py:165: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  input_sheet.update('G1', [[f"類似度: {similarity_score:.4f}"]])


最終結果 (F1) を出力完了。類似度: 0.8999
